# Imports

In [2]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import random


# Question 1: Small World Phenomena in Random Networks

## Helpers

In [3]:
def generate_1d_lattice(N, k=2):
    edges = []
    half_k = k // 2
    for i in range(N):
        for j in range(1, half_k + 1):
            edges.append((i, (i + j) % N))
            edges.append((i, (i - j) % N))
    G = ig.Graph(edges=edges, n=N, directed=False)
    return G



In [4]:
def generate_2d_lattice(N):
    L = int(np.sqrt(N))
    assert L * L == N, "N must be a perfect square for 2D lattice"

    def idx(x, y):
        return x * L + y

    edges = []
    for x in range(L):
        for y in range(L):
            i = idx(x, y)
            edges.append((i, idx((x + 1) % L, y)))
            edges.append((i, idx(x, (y + 1) % L)))

    G = ig.Graph(edges=edges, n=N, directed=False)
    return G


In [ ]:
def generate_3d_lattice(N):
    L = int(round(N ** (1/3)))
    assert L ** 3 == N, "N must be a perfect cube for 3D lattice"

    def idx(x, y, z):
        return x * L * L + y * L + z

    edges = []
    for x in range(L):
        for y in range(L):
            for z in range(L):
                i = idx(x, y, z)
                edges.append((i, idx((x + 1) % L, y, z)))
                edges.append((i, idx(x, (y + 1) % L, z)))
                edges.append((i, idx(x, y, (z + 1) % L)))

    G = ig.Graph(edges=edges, n=N, directed=False)
    return G


In [6]:
def generate_random_network(N, k_avg=4):
    p = k_avg / (N - 1)
    return ig.Graph.Erdos_Renyi(n=N, p=p, directed=False)


In [7]:
def average_shortest_path(G, sample_size=200):
    N = G.vcount()
    nodes = random.sample(range(N), min(sample_size, N))

    total_dist = 0
    count = 0

    for v in nodes:
        dists = G.shortest_paths(v)[0]
        for d in dists:
            if d > 0 and np.isfinite(d):
                total_dist += d
                count += 1

    return total_dist / count


## Simulation

In [ ]:
Ns = np.logspace(np.log10(500), np.log10(5000), num=6, dtype=int)
results = {'1D': [], '2D': [], '3D': [], 'RN': []}

for N in Ns:
    results['1D'].append(
        average_shortest_path(generate_1d_lattice(N), sample_size=N)
    )
    results['2D'].append(
        average_shortest_path(generate_2d_lattice(N), sample_size=N)
    )
    results['3D'].append(
        average_shortest_path(generate_3d_lattice(N), sample_size=N)
    )

    # Random network: average over 5 realizations
    rn_vals = []
    for _ in range(5):
        rn_vals.append(
            average_shortest_path(generate_random_network(N), sample_size=N)
        )
    results['RN'].append(np.mean(rn_vals))


C:\Users\Admin\AppData\Local\Temp\ipykernel_8116\2417551432.py:9: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  dists = G.shortest_paths(v)[0]


AssertionError: N must be a perfect square for 2D lattice